Fuck it, gonna do it myself

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from typing import List, Union, Optional
from functools import partial
import copy

import itertools

In [20]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

In [2]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (HookedRootModule, HookPoint)
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from datasets import load_dataset

In [3]:
model = HookedTransformer.from_pretrained('pythia-70m')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


In [4]:
ds = load_dataset("wikimedia/wikipedia", "20231101.en")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
len(ds['train']['text'])

6407814

In [6]:
rows = 10
activation_store = []

In [7]:
for i in tqdm.trange(rows):
    _, cache = model.run_with_cache(ds['train']['text'][i], names_filter=['blocks.4.hook_resid_pre'])
    activations = cache['blocks.4.hook_resid_pre'][0]
    activations = activations[torch.randperm(activations.shape[0])]
    activation_store.append(activations)

  0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
activation_store = torch.cat(activation_store, dim=0)
# store 
torch.save(activation_store, 'activation_store.pt')

In [13]:
activation_store = torch.load('activation_store.pt')
activation_store.shape

torch.Size([18988, 512])

In [14]:
class AutoEncoder(nn.Module):
    def __init__(self, hidden_size=4096, device='cuda'):
        super().__init__()
        self.l1_coeff = 3e-4
        self.d_model = 512
        self.W_enc = nn.Linear(self.d_model, hidden_size)
        self.W_dec = nn.Linear(hidden_size, self.d_model)
        self.b_enc = nn.Parameter(torch.zeros(hidden_size))
        self.b_dec = nn.Parameter(torch.zeros(self.d_model))
        self.W_dec.weight.data = self.W_dec.weight.data / self.W_dec.weight.data.norm(dim=-1, keepdim=True)
        self.to(device)

    def forward(self, x):
        x_cent = x - self.b_dec
        acts = F.relu(self.W_enc(x_cent) + self.b_enc)
        x_hat = self.W_dec(acts) + self.b_dec
        l2_loss = (x - x_hat).pow(2).sum(-1).mean(0)
        l1_loss = self.l1_coeff * (acts.float().abs().sum())
        loss = l2_loss + l1_loss
        return loss, x_hat, acts, l2_loss, l1_loss
    
    @torch.no_grad()
    def make_decoder_weights_and_grad_unit_norm(self):
        W_dec_normed = self.W_dec.weight.data / self.W_dec.weight.data.norm(dim=-1, keepdim=True)
        W_dec_grad_proj = (self.W_dec.weight.grad * W_dec_normed).sum(-1, keepdim=True) * W_dec_normed
        self.W_dec.weight.grad -= W_dec_grad_proj
        self.W_dec.data = W_dec_normed

In [21]:
def plot_losses(sae_l1_weights, sae_train_losses):
    # make two side-by-side plots
    fig = make_subplots(rows=1, cols=2, subplot_titles=('L1 weights', 'Train losses'))
    fig.add_trace(go.Scatter(y=sae_l1_weights), row=1, col=1)
    fig.add_trace(go.Scatter(y=sae_train_losses), row=1, col=2)
    fig.update_layout(title='Autoencoder training progress')
    fig.show()

In [26]:
sae = AutoEncoder()
sae_train_losses = []
sae_l1_weights = []
epochs = 10
batch_size = 64
optimizer = optim.Adam(sae.parameters(), lr=1e-4)

In [27]:
for epoch in tqdm.trange(epochs):
    epoch_loss = 0
    epoch_l1_weight = 0
    for i in range(0, len(activation_store), batch_size):
        batch = activation_store[i:i+batch_size]
        loss, _, _, l2_loss, l1_loss = sae(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_l1_weight += sae.W_dec.weight.abs().mean().item()
    sae_train_losses.append(epoch_loss / (len(activation_store) / batch_size))
    sae_l1_weights.append(epoch_l1_weight / (len(activation_store) / batch_size))
    plot_losses(sae_l1_weights, sae_train_losses)
    clear_output(wait=True)

In [ ]:
for epoch in tqdm.trange(epochs):
    epoch_loss = 0
    for acts in activation_store:
        loss, x_hat, acts_hat, l2_loss, l1_loss = sae(acts)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        sae.make_decoder_weights_and_grad_unit_norm()
        optimizer.zero_grad()
    l1_weights = sae.W_enc.weight.abs().sum()
    sae_l1_weights.append(l1_weights.item())
    sae_train_losses.append(epoch_loss / rows)

    plot_losses(sae_l1_weights, sae_train_losses)
    clear_output(wait=True)

In [40]:
fig = px.line(sae_train_losses, title='SAE Training Loss')
fig.show()

In [ ]:
sample_text = ds['train']['text'][0]